In [181]:
from sklearn import datasets
import pandas as pd
from int_met import *

In [147]:
x = datasets.load_breast_cancer()
#x=pd.read_csv('Breast cancer.csv')  

In [148]:
data = x.data
feature_names = x.feature_names
y = x.target

In [149]:
import numpy as np
from sklearn.ensemble import RandomForestClassifier
from sklearn.tree import DecisionTreeClassifier
model = RandomForestClassifier()
#model=DecisionTreeClassifier()


In [150]:
train_data = data[1:300,:]
train_labels=y[1:300]
test_data = data[301:505,:]
test_labels=y[301:505]

In [151]:
model = model.fit(train_data,y=train_labels)

In [152]:
from sklearn.metrics import f1_score
y_pred = model.predict(test_data)
error = f1_score(test_labels,y_pred) 
error
y_pred

array([1, 0, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 0, 1, 1, 1, 0, 1,
       0, 1, 1, 1, 1, 0, 0, 0, 1, 1, 1, 1, 0, 1, 0, 1, 0, 0, 1, 1, 0, 1,
       1, 1, 1, 1, 1, 1, 0, 0, 0, 1, 1, 1, 1, 1, 1, 1, 1, 1, 0, 1, 0, 0,
       1, 0, 0, 0, 1, 0, 0, 1, 0, 1, 1, 1, 0, 1, 1, 1, 1, 1, 1, 1, 1, 1,
       0, 1, 1, 0, 0, 1, 1, 1, 1, 1, 1, 0, 1, 1, 1, 1, 1, 0, 1, 0, 1, 1,
       1, 1, 0, 1, 1, 1, 0, 1, 1, 1, 0, 1, 1, 1, 1, 1, 1, 1, 1, 0, 1, 0,
       0, 1, 0, 1, 1, 1, 1, 1, 0, 1, 1, 0, 1, 0, 1, 1, 0, 1, 0, 1, 1, 1,
       1, 1, 1, 1, 1, 0, 0, 1, 1, 1, 1, 1, 1, 0, 1, 1, 1, 1, 1, 1, 1, 0,
       1, 1, 0, 1, 1, 1, 1, 0, 1, 1, 0, 1, 0, 1, 0, 0, 1, 1, 1, 1, 1, 0,
       0, 1, 0, 1, 0, 1])

In [153]:
#necessary imports
from pyBreakDown.explainer import Explainer
from pyBreakDown.explanation import Explanation

In [154]:
exp = Explainer(clf=model, data=train_data, colnames=feature_names)

In [155]:
def run_PyBreakDown(model,data,observations,colnames):
    explanations=[]
    y=[]
    from pyBreakDown.explainer import Explainer
    from pyBreakDown.explanation import Explanation
    from math import exp as e
    exp = Explainer(clf=model, data=data, colnames=feature_names)
    if observations.ndim==1:
        explanation = exp.explain(observation=observations,direction="up")
        pred=explanation._final_prediction
        pred=round((e(pred)/(1+e(pred))))
        y.append(pred)
        explanations.append(explanation)
    else:
        for i in range(0,len(observations)):
            observation = observations[i]
            explanation = exp.explain(observation=observation,direction="up")
            pred=explanation._final_prediction
            pred=round((e(pred)/(1+e(pred))))
            y.append(pred)
            explanations.append(explanation)
    return explanations,y

In [156]:
def get_Coefs(explanations):
    coefs=[]
    for explanation in explanations:
        coef=[]
        for feature in explanation._attributes:
            coef.append(feature.contribution)
        coefs.append(coef)
    return coefs

In [157]:
explanations1,y1 = run_PyBreakDown(model=model,data=train_data,observations=data[567:569,:],colnames=feature_names)
explanations2,y2 = run_PyBreakDown(model=model,data=train_data,observations=data[567:569,:],colnames=feature_names)

In [158]:
coefs1 = get_Coefs(explanations1)
coefs2 = get_Coefs(explanations2)



In [191]:
calc_identity(coefs1,coefs2)
np.asarray(coefs1

[[0.0,
  -0.0033444816053511683,
  -0.03678929765886285,
  -0.4381270903010034,
  -0.03678929765886288,
  0.0,
  0.0,
  0.0,
  0.0,
  0.0,
  0.0,
  0.0,
  0.0,
  0.0,
  0.0,
  0.0,
  0.0,
  0.0,
  0.0,
  0.0,
  0.0,
  0.0,
  0.0,
  0.0,
  0.0,
  0.0,
  0.0,
  0.0,
  0.0,
  0.0,
  0.0],
 [1.0,
  0.0033444816053511683,
  0.0,
  0.0033444816053511683,
  0.1070234113712375,
  0.1605351170568562,
  0.14381270903010035,
  0.0635451505016722,
  0.0,
  0.0033444816053511683,
  0.0,
  0.0,
  0.0,
  0.0,
  0.0,
  0.0,
  0.0,
  0.0,
  0.0,
  0.0,
  0.0,
  0.0,
  0.0,
  0.0,
  0.0,
  0.0,
  0.0,
  0.0,
  0.0,
  0.0,
  0.0]]

In [189]:
def calc_stability2(exp, labels):
    total = labels.shape[0]
    label_values = np.unique(labels)
    print("labels", labels)

    print("label_values", label_values)
    n_clusters = label_values.shape[0]
    init = np.array([[np.average(exp[np.where(labels == i)], axis = 0)] for i in label_values]).squeeze()
    ct = sklearn.cluster.KMeans(n_clusters = n_clusters, n_jobs=5, random_state=1, n_init=10, init = init)
    ct.fit(exp)
    error = np.sum(np.abs(labels-ct.labels_))
    if error/total > 0.5:
        error = total-error
    return error, total

In [190]:
y_test=y[567:569]
from int_met import *
y_test=np.asarray(y_test)
calc_stability2(coefs1, y_test)
y_test

labels [0 1]
label_values [0 1]


TypeError: list indices must be integers or slices, not tuple

In [173]:
y_test

array([0, 1])

In [161]:
x2 = datasets.load_boston()

In [162]:
x2.target[302]

26.4

In [163]:
dir(explanation)

['__bool__',
 '__class__',
 '__delattr__',
 '__dir__',
 '__doc__',
 '__eq__',
 '__format__',
 '__ge__',
 '__getattribute__',
 '__gt__',
 '__hash__',
 '__init__',
 '__init_subclass__',
 '__le__',
 '__lt__',
 '__ne__',
 '__new__',
 '__reduce__',
 '__reduce_ex__',
 '__repr__',
 '__setattr__',
 '__sizeof__',
 '__str__',
 '__subclasshook__']

In [ ]:
y_test=np.asarray(y_test)